# Имитационное обучение:  алгоритм SQIL.

Научиться имитировать поведение экспертов из демонстраций может быть непросто, особенно в средах с многомерными непрерывными наблюдениями и неизвестной динамикой. Методы копирования поведения, страдают от сдвига распределения (distribution shift): поскольку агент жадно имитирует продемонстрированные действия, он может отклоняться от продемонстрированных состояний, что приводит к накоплению ошибок.

Оригинальная статья: [SQIL: Imitation Learning via Reinforcement Learning with Sparse Rewards](https://arxiv.org/abs/1905.11108)

В данной части мы попробуем применить идею из этой статьи для среды ``LunarLanderContinuous-v2``, воспользовавшись кодом алгоритма DDPG с семинара.

In [ ]:
# @title Установка зависимостей
try:
    import google.colab
    COLAB = True
except ModuleNotFoundError:
    COLAB = False
    pass

if COLAB:
    !apt -qq update -y
    !apt -qq install swig -y
    !pip -q install box2d-py
    !pip -q install "gymnasium[classic-control, box2d, atari, accept-rom-license]"
    !pip -q install piglet
    !pip -q install imageio_ffmpeg
    !pip -q install moviepy==1.0.3

In [ ]:
# @title Импортирование зависимостей
import math
import random

import gymnasium as gym
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.distributions import Normal

# библиотеки и функции, которые потребуеются для показа видео
import glob
import io
import base64
from IPython import display as ipythondisplay
from IPython.display import HTML
import matplotlib.pyplot as plt
from gymnasium.wrappers import RecordVideo

%matplotlib inline

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)


def show_video(folder="./video"):
    mp4list = glob.glob(folder + '/*.mp4')
    if len(mp4list) > 0:
        mp4 = sorted(mp4list, key=lambda x: x[-15:], reverse=True)[0]
        video = io.open(mp4, 'r+b').read()
        encoded = base64.b64encode(video)
        ipythondisplay.display(HTML(data='''<video alt="test" autoplay
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
    else:
        print("Could not find video")


def show_progress(rewards_batch, log, reward_range=None):
    """
    Удобная функция, которая отображает прогресс обучения.
    """

    if reward_range is None:
        reward_range = [-990, +10]
    mean_reward = np.mean(rewards_batch)
    log.append([mean_reward])

    clear_output(True)
    plt.figure(figsize=[8, 4])
    plt.subplot(1, 2, 1)
    plt.plot(list(zip(*log))[0], label='Mean rewards')
    plt.legend(loc=4)
    plt.grid()
    plt.grid()
    plt.show()

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

print(device)

# Нормализация действия и добавление шума:

In [ ]:
class NormalizedActions(gym.ActionWrapper):

    def action(self, action):
        low_bound = self.action_space.low
        upper_bound = self.action_space.high
        # [L, M, R]
        # actions are in [-1, 1]
        # Нормализуйте действия
        ####### Здесь ваш код ########
        action = 
        ##############################
        action = np.clip(action, low_bound, upper_bound)

        return action

    def reverse_action(self, action):
        pass


class GaussNoise:

    def __init__(self, sigma):
        super().__init__()

        self.sigma = sigma

    def get_action(self, action):
        # добавьте нормальный шум
        ####### Здесь ваш код ########
        noisy_action = 
        ##############################
        return noisy_action

# Value и Policy сети:

In [ ]:
class ValueNetwork(nn.Module):
    def __init__(
            self,
            num_inputs,
            num_actions,
            hidden_size,
    ):
        super().__init__()
        # добавьте 3 линейных слой (не забудьте про функцию активации Tanh)
        ####### Здесь ваш код ########
        self.net = 
        ##############################

    def forward(self, state, action):
        x = torch.cat([state, action], 1)
        x = self.net(x)
        return x


class PolicyNetwork(nn.Module):
    def __init__(
            self,
            num_inputs,
            num_actions,
            hidden_size,
    ):
        super().__init__()

        # определяем граф вычисления для Policy Network
        # добавьте 3 линейных слой (не забудьте про функцию активации Tanh)
        ####### Здесь ваш код ########
        self.net =
        ##############################

    def forward(self, state):
        # определяем прямой проход по графу вычислений
        # x =
        x = state
        x = self.net(x)
        return x

    def get_action(self, state):
        """
        функция для выбора действия
        """
        state = torch.tensor(state, dtype=torch.float32).unsqueeze(0).to(device)
        action = self.forward(state)
        action = action.detach().cpu().numpy()[0]
        action = np.clip(action, -1.0, 1.0)

        return action

# DDPG обновление

<img src="https://spinningup.openai.com/en/latest/_images/math/5811066e89799e65be299ec407846103fcf1f746.svg">

Оригинальная статья:  <a href="https://arxiv.org/abs/1509.02971">Continuous control with deep reinforcement learning Arxiv</a>

In [ ]:
def ddpg_update(
        state,
        action,
        reward,
        next_state,
        done,
        gamma=0.99,
        min_value=-np.inf,
        max_value=np.inf,
        soft_tau=0.001,
):
    state = torch.tensor(state, dtype=torch.float32).to(device)
    next_state = torch.tensor(next_state, dtype=torch.float32).to(device)
    action = torch.tensor(action, dtype=torch.float32).to(device)
    reward = torch.tensor(reward, dtype=torch.float32).unsqueeze(1).to(device)
    done = torch.tensor(np.float32(done)).unsqueeze(1).to(device)

    # считаем policy loss по формуле выше, используя value_net и policy_net
    ####### Здесь ваш код ########
    policy_loss = 
    ##############################

    next_action = target_policy_net(next_state)
    target_value = target_value_net(next_state, next_action.detach())
    # считаем таргет Q функцию
    ####### Здесь ваш код ########
    expected_value = 
    ##############################
    expected_value = torch.clamp(expected_value, min_value, max_value)

    value = value_net(state, action)
    value_loss = nn.MSELoss()(value, expected_value.detach())


    policy_optimizer.zero_grad()
    policy_loss.backward()
    policy_optimizer.step()

    value_optimizer.zero_grad()
    value_loss.backward()
    value_optimizer.step()

    for target_param, param in zip(target_value_net.parameters(), value_net.parameters()):
        target_param.data.copy_(
            target_param.data * (1.0 - soft_tau) + param.data * soft_tau
        )

    for target_param, param in zip(target_policy_net.parameters(), policy_net.parameters()):
        target_param.data.copy_(
            target_param.data * (1.0 - soft_tau) + param.data * soft_tau
        )

# Стандартная и комбинированная память прецедентов:  

In [ ]:
class ReplayBuffer:
    def __init__(self, capacity):
        self.capacity = capacity
        self.buffer = []
        self.position = 0

    def push(self, state, action, reward, next_state, done):
        if len(self.buffer) < self.capacity:
            self.buffer.append(None)
        self.buffer[self.position] = (state, action, reward, next_state, done)
        self.position = (self.position + 1) % self.capacity

    def sample(self, batch_size):
        batch = random.sample(self.buffer, batch_size)
        state, action, reward, next_state, done = map(np.stack, zip(*batch))
        return state, action, reward, next_state, done

    def __len__(self):
        return len(self.buffer)


class CombinedReplayBuffer:
    def __init__(self, capacity):
        self.capacity = capacity
        self.demo = ReplayBuffer(self.capacity)
        self.agent = ReplayBuffer(self.capacity)

    def push_demo(self, state, action, reward, next_state, smart_done):
        # модифицируем вознаграждения, как этого предусматривает алгоритм SQIL
        # reward =
        ####### Здесь ваш код ########
        reward =
        ##############################
        self.demo.push(state, action, reward, next_state, smart_done)

    def push(self, state, action, reward, next_state, smart_done):
        # модифицируем вознаграждения, как этого предусматривает алгоритм SQIL
        # reward =
        ####### Здесь ваш код ########
        reward = 
        ##############################
        self.agent.push(state, action, reward, next_state, smart_done)


    def sample(self, batch_size):
        demo_batch_size = min(batch_size // 2, len(self.demo))
        # набираем данные из обоих буферов (семплирование из буферов)
        ####### Здесь ваш код ########

        ##############################

        return np.concatenate([states, demo_states]), \
               np.concatenate([actions, demo_actions]), \
               np.concatenate([rewards, demo_rewards]), \
               np.concatenate([next_states, demo_next_states]), \
               np.concatenate([dones, demo_dones])

    def __len__(self):
        return len(self.demo) + len(self.agent)

 # Метод ``generate_session``


In [ ]:
def generate_session(train=False):
    """эпизод взаимодействие агента со средой, а также вызов процесса обучения"""
    total_reward = 0
    state, info = env.reset()

    done = False
    while not done:
        action = policy_net.get_action(state)
        if train:
            action = noise.get_action(action)
        next_state, reward, term, trunc, info = env.step(action)
        done = term or trunc
        if train:
            replay_buffer.push(state, action, reward, next_state, term)
            if len(replay_buffer) > replay_buffer_size + 1500:
                ddpg_update(*replay_buffer.sample(batch_size))
        total_reward += reward
        state = next_state
        if done:
            break

    return total_reward

# Задание гиперпараметров и инициализация всего и вся:

In [ ]:
env_name = "LunarLanderContinuous-v3"

max_steps = 350
env = NormalizedActions(gym.make(env_name, max_episode_steps=max_steps))

noise = GaussNoise(sigma=0.1)

state_dim = env.observation_space.shape[0]
action_dim = env.action_space.shape[0]
hidden_dim = 512

value_net = ValueNetwork(state_dim, action_dim, hidden_dim).to(device)
policy_net = PolicyNetwork(state_dim, action_dim, hidden_dim).to(device)

target_value_net = ValueNetwork(state_dim, action_dim, hidden_dim).to(device)
target_policy_net = PolicyNetwork(state_dim, action_dim, hidden_dim).to(device)

for target_param, param in zip(target_value_net.parameters(), value_net.parameters()):
    target_param.data.copy_(param.data)

for target_param, param in zip(target_policy_net.parameters(), policy_net.parameters()):
    target_param.data.copy_(param.data)

value_lr = 1e-3
policy_lr = 1e-4

value_optimizer = optim.Adam(value_net.parameters(), lr=value_lr, weight_decay=1e-6)
policy_optimizer = optim.Adam(policy_net.parameters(), lr=policy_lr, weight_decay=1e-6)

batch_size = 128

### Генерация экспертных данных:

In [ ]:
from gymnasium.envs.box2d.lunar_lander import heuristic

replay_buffer_size = 100000
replay_buffer = CombinedReplayBuffer(replay_buffer_size)

noise = GaussNoise(sigma=0.1)
episodes = 0
while len(replay_buffer) < replay_buffer_size:
    episodes += 1
    done = False
    state, _ = env.reset()
    episode_reward = 0
    while not done:
        action = noise.get_action(heuristic(env, state))
        next_state, reward, terminated, truncated, _ = env.step(action)
        replay_buffer.push_demo(state, action, reward, next_state, terminated)
        done = terminated or truncated
        episode_reward += reward
        state = next_state
    if episodes % 100 == 0:
        print(f"episode: {episodes}, reward: {episode_reward}, replay_size:", len(replay_buffer))
env.close()

In [ ]:
env = NormalizedActions(gym.make(env_name, max_episode_steps=max_steps))
noise = GaussNoise(sigma=0.001)
valid_mean_rewards = []
for i in range(100):
    session_rewards_train = [generate_session(train=True) for _ in range(10)]

    mean_reward = np.mean(session_rewards_train)
    print(f"epoch #{i:02d}\tmean reward (train) = {mean_reward:.3f}\t")

    if mean_reward > 200:
        print("Выполнено!")
        break

env.close()

# Посмотрим за полетом:

In [ ]:
env = NormalizedActions(gym.make(env_name, max_episode_steps=max_steps, render_mode='rgb_array'))
env = RecordVideo(env, f"./video")

done = False

state, info = env.reset()

while not done:
    action = policy_net.get_action(state)
    state, _, term, trunc, info = env.step(action)
    done = term or trunc

env.close()
show_video()

# Загрузка датасета MuJoCo

Загрузите датасет для одной из сред MuJoCo. Используйте [данные](https://minari.farama.org/main/datasets/mujoco/) от Minari.

# Обучение на загруженных данных

Повторите обучение алгоритма SQIL на загруженном датасете. Необходимо обучить алгоритм на каждом из вариантов датасета (simple, medium, expert).

Проанализируйте полученные результаты обучения на каждом из датасетов.